In [20]:
import polars as pl
import pandas as pd
import preprocessing.transform_data as td

In [2]:
df_raw = pl.read_parquet("./data/2022_ST4000.parquet")

In [3]:
df_raw = td.filter_out_inconsistent_drives(df_raw)

In [5]:
Train, Test = td.custom_train_test_split(df_raw)

In [11]:
Train

,0,fail
9539,Z305D6JZ,1
13119,Z304KCQW,0
1309,Z302A117,0
5846,Z305GXK8,0
9833,Z305D2WZ,0
...,...,...
7458,Z304R607,0
8095,Z304KD49,0
1981,Z3059J34,0
5712,Z304L8T7,0


In [44]:
serial_nr_train = td.get_serial(df_raw)

In [13]:
# Yield successive n-sized
# chunks from l.
def divide_chunks(l, n):
     
    # looping till length l
    for i in range(0, len(l), n):
        yield l[i:i + n]

In [19]:
train_chunks = divide_chunks(list(Train[0]), 1500)
test_chunks = divide_chunks(list(Test[0]), 1500)

In [18]:
len(list(train_chunks))

10

In [21]:
td.create_chunks(df_raw,list(Train[0]), name="Train")

TypeError: create_chunks() got an unexpected keyword argument 'name'

In [ ]:
counter = 1

for l in test_chunks:
    my_index = pd.MultiIndex(levels=[[],[]],codes=[[],[]], names=["instances","timepoints"])
    df_rocket = pd.DataFrame(columns= td.feature_list(target=False), index= my_index)
    df_rocket = td.transform_all(df_raw,l, df_rocket)
    df_rocket.to_parquet(f"./data/datachunks/test_chunk{counter}.parquet")
    counter += 1

In [ ]:
df_old = pd.DataFrame()
for i in range(1,):
    df_new = pd.read_parquet(f"data/datachunks/chunk{i}.parquet")
    df_old = pd.concat([df_old, df_new])


In [48]:
df_old

smart_1_raw  smart_3_raw  smart_4_raw  smart_5_raw  \
instances timepoints                                                       
0         0           118888896.0          0.0         14.0          0.0   
          1             3300704.0          0.0         14.0          0.0   
          2             4250832.0          0.0         14.0          0.0   
          3            23049648.0          0.0         14.0          0.0   
          4           151092552.0          0.0         14.0          0.0   
...                           ...          ...          ...          ...   
25098     9            50690312.0          0.0         10.0          0.0   
          10          156343176.0          0.0         10.0          0.0   
          11          190132800.0          0.0         10.0          0.0   
          12           45518888.0          0.0         10.0          0.0   
          13           77442552.0          0.0         10.0          0.0   

                      smart_7_raw  smart_9_raw  smart_10_raw  smart_12_raw  \
instances timepoints                                                         
0         0           299906602.0      53960.0           0.0          14.0   
          1           300743900.0      53984.0           0.0          14.0   
          2           302415758.0      54008.0           0.0          14.0   
          3           305432802.0      54032.0           0.0          14.0   
          4           306764525.0      54056.0           0.0          14.0   
...                           ...          ...           ...           ...   
25098     9           718217009.0      60555.0           0.0          10.0   
          10          719402620.0      60579.0           0.0          10.0   
          11          721080189.0      60603.0           0.0          10.0   
          12          722565636.0      60626.0           0.0          10.0   
          13          723662392.0      60650.0           0.0          10.0   

                      smart_187_raw  smart_188_raw  ...  smart_188_normalized  \
instances timepoints                                ...                         
0         0                     0.0            0.0  ...                 100.0   
          1                     0.0            0.0  ...                 100.0   
          2                     0.0            0.0  ...                 100.0   
          3                     0.0            0.0  ...                 100.0   
          4                     0.0            0.0  ...                 100.0   
...                             ...            ...  ...                   ...   
25098     9                     0.0            0.0  ...                 100.0   
          10                    0.0            0.0  ...                 100.0   
          11                    0.0            0.0  ...                 100.0   
          12                    0.0            0.0  ...                 100.0   
          13                    0.0            0.0  ...                 100.0   

                      smart_192_normalized  smart_193_normalized  \
instances timepoints                                               
0         0                          100.0                  93.0   
          1                          100.0                  93.0   
          2                          100.0                  93.0   
          3                          100.0                  93.0   
          4                          100.0                  93.0   
...                                    ...                   ...   
25098     9                          100.0                  98.0   
          10                         100.0                  98.0   
          11                         100.0                  98.0   
          12                         100.0                  98.0   
          13                         100.0                  98.0   

                      smart_194_normalized  smart_197_normalized  \
instances t

In [62]:
y = []
for instance in df_old.index.levels[0]:
    if 1.0 in list(df_old.loc[(instance),"failing_in14days"]):
        y.append(1)
    else: 
        y.append(0)

In [64]:
y.count(1)

49

In [58]:
df_old.loc[(0),"failing_in14days"]

timepoints
0     0.0
1     0.0
2     0.0
3     0.0
4     0.0
5     0.0
6     0.0
7     0.0
8     0.0
9     0.0
10    0.0
11    0.0
12    0.0
13    0.0
Name: failing_in14days, dtype: float64

In [50]:
df_old.index.levels[0]
df.loc[('at', [1,3,4]), 'Dwell'

Int64Index([    0,     1,     2,     3,     4,     5,     6,     7,     8,
                9,
            ...
            25089, 25090, 25091, 25092, 25093, 25094, 25095, 25096, 25097,
            25098],
           dtype='int64', name='instances', length=25099)

In [ ]:
# code courtesy of jezrael on stackoverflow (https://stackoverflow.com/questions/46445965/reset-a-recurring-multiindex-in-pandas)
a1 = df_old.index.get_level_values(0).to_series()
a2 = a1.ne(a1.shift()).cumsum() - 1

mux = pd.MultiIndex.from_arrays([a2, df_old.index.get_level_values(1)], names=df_old.index.names)

df_old.index = mux

In [ ]:
df_old

In [ ]:
from sktime.transformations.panel.rocket import Rocket
from sktime.datatypes import convert,convert_to,mtype,check_raise

In [ ]:
rocket = Rocket()

In [49]:
col_list = td.feature_list(target=False)
df_rocket = df_old[col_list]

In [ ]:
X_train2 = convert(df_rocket,from_type="pd-multiindex", to_type="df-list")

In [ ]:
type(X_train2[0])

In [ ]:
rocket.fit(X_train2)

In [ ]:
transformed_X = rocket.transform(X_train2)

In [ ]:
transformed_X